In [1]:

import os
import json
from pypelines.sklearn_pypeline import SklearnPipeline

In [2]:
# code output
skl_pypelines = SklearnPipeline(data = "titanic",target = 'Survived'
                            , model_type = 'classification'
                            , models = ['GaussianNB Classifier','Logistic Regression','Random Forest']
                            , nfolds = 5, output_format='code')


In [3]:
skl_pypelines.model_list()

['GBT Classifier', 'Logistic Regression']


In [4]:
skl_pypelines.get_code()


from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import plotly.express as px
import plotly.graph_objects as go


# target dataframe: titanic
titanic = pd.read_csv("./titanic.csv")
target = "Survived"
features = list(titanic.columns.drop("Survived"))
feature_df = titanic[features]

# get numerical and categorical columns
bool_cols = feature_df.select_dtypes(include=['bool']).columns.tolist()
titanic[bool_cols] = feature_df[bool_cols].astype(int)
numerical_cols = feature_df.select_dtypes(include=['int', 'float']).columns.tolist()
categorical_cols = feature_df.select_dtypes(include=['object']).columns.tolist(

In [ ]:
# script output
skl_pypelines = SklearnPipeline(data = "titanic",target = 'Survived'
                            , model_type = 'classification'
                            , models = ['GaussianNB Classifier','Logistic Regression','Random Forest']
                            , nfolds = 5, output_format='script',
                            output_folder = "./code_output")

In [ ]:
skl_pypelines.model_list()

In [ ]:
skl_pypelines.get_code()